# 🔧 PASO 1 – Preparar dataset

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Crear función de preparación con lags y clase
def preparar_dataset_para_regresion(df):
    df = df.sort_values(['product_id', 'periodo'])
    
    # Generar lags tn_1 a tn_11 por producto
    for i in range(1, 12):
        df[f'tn_{i}'] = df.groupby('product_id')['tn'].shift(i)
    
    # Crear campo "clase" con tn en periodo+2
    df['clase'] = df.groupby('product_id')['tn'].shift(-2)
    
    return df

In [ ]:
df = pd.read_csv("../../data/raw/sell-in.csv", sep='\t')
df = df.groupby(['periodo','product_id']).agg({'tn':'sum'})
df = preparar_dataset_para_regresion(df)

,periodo\tcustomer_id\tproduct_id\tplan_precios_cuidados\tcust_request_qty\tcust_request_tn\ttn
0,201701\t10234\t20524\t0\t2\t0.053\t0.053
1,201701\t10032\t20524\t0\t1\t0.13628\t0.13628
2,201701\t10217\t20524\t0\t1\t0.03028\t0.03028
3,201701\t10125\t20524\t0\t1\t0.02271\t0.02271
4,201701\t10012\t20524\t0\t11\t1.54452\t1.54452
...,...
2945813,201912\t10105\t20853\t0\t1\t0.0223\t0.0223
2945814,201912\t10092\t20853\t0\t1\t0.00669\t0.00669
2945815,201912\t10006\t20853\t0\t7\t0.02898\t0.02898
2945816,201912\t10018\t20853\t0\t4\t0.01561\t0.01561


# 📚 PASO 2 – Entrenar regresión lineal

In [ ]:
# Lista de productos mágicos
magicos = [
    20002, 20003, 20006, 20010, 20011, 20018, 20019, 20021,
    20026, 20028, 20035, 20039, 20042, 20044, 20045, 20046, 20049,
    20051, 20052, 20053, 20055, 20008, 20001, 20017, 20086, 20180,
    20193, 20320, 20532, 20612, 20637, 20807, 20838
]

# Entrenar regresión lineal en periodo base con productos mágicos
def entrenar_regresion_lineal(df, periodo_base, magicos,  modelo='ridge', alpha=1.0, random_state=42):
    columnas_tn = ['tn'] + [f'tn_{i}' for i in range(1, 12)]
    
    # Filtrar registros del periodo base y productos mágicos
    df_train = df[(df['periodo'] == periodo_base) & (df['product_id'].isin(magicos))].copy()
    
    # Eliminar filas con datos faltantes
    df_train = df_train.dropna(subset=columnas_tn + ['clase'])

    # Entrenar modelo
    X = df_train[columnas_tn]
    y = df_train['clase']

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    if modelo == 'ridge':
        reg = Ridge(alpha=alpha, random_state=random_state)
    elif modelo == 'lasso':
        reg = Lasso(alpha=alpha, random_state=random_state, max_iter=5000)
    else:
        reg = LinearRegression()
    
    reg.fit(X_scaled, y)
    y_pred = reg.predict(X_scaled)
    
    
    # Mostrar coeficientes
    coef = pd.Series([reg.intercept_] + reg.coef_.tolist(), index=['intercept'] + columnas_tn)
    return reg, scaler, coef.sort_values(ascending=False)



    

   


# 🔮 PASO 3 – Predecir con el modelo entrenado

In [ ]:
# Predecir para un periodo futuro
def predecir_regresion(df, model, periodo_a_predecir):
    columnas_tn = ['tn'] + [f'tn_{i}' for i in range(1, 12)]
    
    df_pred = df[df['periodo'] == periodo_a_predecir].copy()
    
    # Separar registros completos e incompletos
    completos = df_pred.dropna(subset=columnas_tn).copy()
    incompletos = df_pred[~df_pred.index.isin(completos.index)].copy()
    
    # Predecir usando regresión lineal
    completos['pred'] = model.predict(completos[columnas_tn])
    
    # Asignar promedio para los incompletos
    pred_media = completos['pred'].mean()
    incompletos['pred'] = pred_media

    # Unir resultados
    df_final = pd.concat([completos, incompletos]).sort_index()
    
    return df_final[['product_id', 'periodo'] + columnas_tn + ['pred']]


# ▶️ PASO 4 – Ejecutar todo el flujo

In [ ]:
# Paso A: preparar dataset (con tus datos originales en df)
df_prep = preparar_dataset_para_regresion(df)

# Paso B: entrenar el modelo con periodo 201812
modelo, coeficientes = entrenar_regresion_lineal(df_prep, periodo_base=201812, magicos=magicos)
print("Coeficientes encontrados:")
print(coeficientes)

# Paso C: predecir para registros del periodo 201912 → objetivo: predecir 202002
df_predicho = predecir_regresion(df_prep, modelo, periodo_a_predecir=201912)

# Paso D: ver primeras predicciones
print(df_predicho[['product_id', 'periodo', 'pred']].head())

# 💾 PASO 5 – Exportar a CSV (opcional)

In [ ]:
# Exportar archivo para enviar a Kaggle (ajustar columnas si necesario)
df_predicho[['product_id', 'pred']].to_csv("predicciones_regresion_lineal.csv", index=False)